## E7. 프로젝트 제출

### 1) 데이터 준비와 전처리

In [1]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'star'}, inplace=True)

In [4]:
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### 2) 분석해봅시다.

In [5]:
# - ratings에 있는 유니크한 영화 개수
# - rating에 있는 유니크한 사용자 수
# - 가장 인기있는 영화 30개(인기순)
ratings.nunique()

user_id        6039
movie_id       3628
star              3
timestamp    412911
dtype: int64

In [6]:
famous_movie = ratings.groupby('movie_id')['user_id'].count()
famous_movie.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

### 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [10]:
my_favorite = [1196, 2997, 356, 2396, 1240]
my_playlist = pd.DataFrame({'user_id': ['kkw']*5, 'movie_id': my_favorite, 'star': [5]*5})
# ['user_id', 'movie_id', 'rating', 'timestamp']
if not ratings.isin({'user_id':['kkw']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)
    
ratings.tail(10)

,user_id,movie_id,star,timestamp
0,6039,3628,5,NaN
1,6039,3629,5,NaN
2,6039,3630,5,NaN
3,6039,3631,5,NaN
4,6039,3632,5,NaN
0,kkw,1196,5,NaN
1,kkw,2997,5,NaN
2,kkw,356,5,NaN
3,kkw,2396,5,NaN
4,kkw,1240,5,NaN


In [11]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [12]:
temp_user_ratings = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_ratings) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_ratings   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_ratings = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_ratings) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_ratings
else:
    print('movie_id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,star,timestamp
0,0,0,5,978300760.0
1,0,1,3,978302109.0
2,0,2,3,978301968.0
3,0,3,4,978300275.0
4,0,4,5,978824291.0
...,...,...,...,...
0,6040,1196,5,NaN
1,6040,2997,5,NaN
2,6040,356,5,NaN
3,6040,2396,5,NaN


### 4) CSR matrix를 직접 만들어 봅시다.

In [13]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_ratings = csr_matrix((ratings.star, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_ratings

<6041x3633 sparse matrix of type '<class 'numpy.longlong'>'
	with 836488 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [14]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [15]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [16]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_ratings_transpose = csr_ratings.T
csr_ratings_transpose

<3633x6041 sparse matrix of type '<class 'numpy.longlong'>'
	with 836488 stored elements in Compressed Sparse Column format>

In [17]:
als_model.fit(csr_ratings_transpose)

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [18]:
kkw, moviiee = user_to_idx['kkw'], movie_to_idx[1240]
kkw_vector, moviiee_vector = als_model.user_factors[kkw], als_model.item_factors[moviiee]

In [19]:
kkw_vector

array([-0.14114699, -0.00968625,  0.26436585, -0.07158237, -0.03034601,
        0.25939474,  0.5473511 , -0.38065296, -0.6283678 ,  0.06286141,
        0.10290731,  0.10709111,  0.32068717, -0.01904651, -0.1900991 ,
       -0.18202917, -0.11245142, -0.12953514,  0.5065762 ,  0.7012125 ,
        0.33064663, -0.09625311,  0.2638329 , -0.9034742 , -0.2910608 ,
        0.04240551, -0.64418143, -0.34187505, -0.4186914 , -0.14473717,
       -0.16145211,  0.6728365 ,  0.20930865, -0.11979603, -0.46151543,
        0.19340192,  0.19739528,  0.08167242, -0.02873948, -0.1168984 ,
       -0.12089578, -0.38670313, -0.22293666,  0.05505537,  0.22312506,
        0.14131665, -0.03303442,  0.12732314, -0.20124061,  0.24328578,
        0.21517047,  0.25025424,  0.3799161 , -0.18249837,  0.20618062,
       -0.00111035, -0.3971548 ,  0.2723403 , -0.11976284, -0.7443592 ,
       -0.0449458 , -0.25058025,  0.29443854, -0.01051862,  0.20819719,
       -0.03319368, -0.42801434,  0.0492595 ,  0.27974832, -0.09

In [20]:
moviiee_vector

array([ 2.42432137e-03,  1.75813539e-03,  2.17608782e-03,  5.53085655e-03,
        1.28571521e-02,  1.23175699e-02,  1.49315577e-02, -2.52648327e-03,
       -6.66090427e-03,  7.79660931e-03,  5.89159410e-03,  2.50696670e-02,
        8.27047415e-03,  1.13445716e-02, -5.86502953e-03, -1.12736365e-02,
        9.35721584e-03,  7.15692295e-05,  1.65600665e-02,  1.17140003e-02,
        8.47406313e-03,  6.05195435e-03,  1.60498619e-02, -4.69162408e-03,
        4.13816702e-03,  9.88070667e-03, -4.64316038e-03, -1.27757723e-02,
        5.65175572e-03,  1.11970473e-02,  5.63415512e-03,  1.97772626e-02,
        1.18986648e-02, -5.07284515e-03,  1.06119765e-02,  1.28845545e-02,
        4.62671230e-03,  2.59857136e-03,  2.00839397e-02, -1.72500452e-03,
        3.59021360e-03,  2.86001177e-03,  2.14639679e-03,  5.05094603e-03,
        1.73144341e-02,  1.39663452e-02,  1.24426577e-02,  1.08423764e-02,
        3.45308939e-03,  1.51251713e-02,  1.51145291e-02,  2.09363867e-02,
        8.49436224e-03,  

In [21]:
np.dot(kkw_vector, moviiee_vector)

0.15819761

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [22]:
favorite_movie = 1111
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(1111, 0.024987759),
 (3421, 0.01905517),
 (3587, 0.019023178),
 (3391, 0.01835682),
 (1538, 0.018304683),
 (3276, 0.018298782),
 (3251, 0.018246409),
 (3339, 0.018147826),
 (2901, 0.018109603),
 (3341, 0.01808886),
 (2608, 0.018033212),
 (3515, 0.017974721),
 (3459, 0.01794609),
 (2964, 0.01793765),
 (3303, 0.01793696)]

In [24]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

[1111,
 3421,
 3587,
 3391,
 1538,
 3276,
 3251,
 3339,
 2901,
 3341,
 2608,
 3515,
 3459,
 2964,
 3303]

In [28]:
def get_similar_movie(movie_name: int):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [29]:
get_similar_movie(1211)

[1211, 1172, 1273, 1207, 1187, 1237, 1575, 1561, 1186, 1206]

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [32]:
user = user_to_idx['kkw']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_ratings, N=20, filter_already_liked_items=True)
movie_recommended

[(779, 0.1822237),
 (835, 0.14660968),
 (542, 0.13492315),
 (1244, 0.13094108),
 (1045, 0.12991872),
 (521, 0.12820736),
 (82, 0.12539637),
 (450, 0.12351746),
 (1186, 0.1218853),
 (1418, 0.11590825),
 (1278, 0.11448597),
 (607, 0.11443292),
 (1474, 0.113648474),
 (812, 0.11360572),
 (1090, 0.109248094),
 (380, 0.10924411),
 (337, 0.10848663),
 (1206, 0.105837926),
 (817, 0.104897425),
 (1188, 0.104887255)]

In [34]:
[idx_to_movie[i[0]] for i in movie_recommended]

[779,
 835,
 542,
 1244,
 1045,
 521,
 82,
 450,
 1186,
 1418,
 1278,
 607,
 1474,
 812,
 1090,
 380,
 337,
 1206,
 817,
 1188]

In [40]:
recommend = movie_to_idx[1186]
explain = als_model.explain(user, csr_ratings, itemid=recommend)
explain

(0.120670864652331,
 [(356, 0.04728150559849643),
  (1196, 0.030429294231113447),
  (1240, 0.021686194038425095),
  (2396, 0.018746861887559536),
  (2997, 0.0025270088967364974)],
 (array([[0.58421844, 0.13400919, 0.11373016, ..., 0.11504031, 0.11763084,
          0.1192164 ],
         [0.07829064, 0.58624009, 0.09007172, ..., 0.095482  , 0.06492121,
          0.07375198],
         [0.06644326, 0.06804454, 0.57571904, ..., 0.04305357, 0.0433837 ,
          0.03938692],
         ...,
         [0.06720867, 0.07139183, 0.04647054, ..., 0.52592705, 0.0181892 ,
          0.05879408],
         [0.06872211, 0.05382303, 0.04420256, ..., 0.07994895, 0.52112392,
          0.00917827],
         [0.06964842, 0.05921246, 0.04287727, ..., 0.10644564, 0.06783531,
          0.5280709 ]]),
  False))

어렵다. 데이터가 두개로 분리되어 있다는 이유 하나만으로 거대한 벽이 세워진 것 같았다.
데이터가 둘로 분리되어 있어서 movie_id를 title로 연결시켜서 불러오는 방법을 구글링해서 찾다가 3)에서 너무 오래 머물러서 우선 뒷부분을 movie_id로 진행했다. 선호하는 영화를 영어 제목으로 입력했었는데, 훈련시키니까 0.000008 이런 식으로 나와서 movie_id 타입에 맞추어 숫자로 입력했다. 그런데 결과가 똑같길래 다시 보니까 '1111' 식으로 입력해서 문제였다. 1111로 작성해 타입을 바꾸었더니 현재의 결과값이 나왔다.
마음 같아서는 다른 잘 하는 사람들 처럼 머지를 이용해서 데이터를 묶어서 영화제목이 바로 나왔으면 좋겠는데, 미뤘다가 하지 않는 내 모습을 보면서 지금 제출하는 게 낫겠다 싶어서 제출하기로 정했다.